<a href="https://colab.research.google.com/github/alangan17/smazc03/blob/main/SMAZC_03_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install yfinance

In [2]:
!pip uninstall gdown -y && pip install gdown
!gdown -V

Found existing installation: gdown 5.1.0
Uninstalling gdown-5.1.0:
  Successfully uninstalled gdown-5.1.0
gdown 5.2.0 at /usr/local/lib/python3.10/dist-packages


In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# 0 Dataset for Modeling: Final Preparations

0.1) Importing data from Drive & defining variable sets


- automated version need to have a daily updated file/database entries




In [4]:
# https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab
# truncated data from Module 2: https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing
!gdown https://drive.google.com/file/d/1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP/view?usp=sharing --fuzzy -O /content/


Downloading...
From (original): https://drive.google.com/uc?id=1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP
From (redirected): https://drive.google.com/uc?id=1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP&confirm=t&uuid=31699872-b1eb-4550-8071-dae6b571712d
To: /content/stocks_df_combined_2024_05_07.parquet.brotli
100% 119M/119M [00:00<00:00, 198MB/s]


In [5]:
# truncated
# df = pd.read_parquet("/content/stocks_df_combined_trunc_2014_2023.parquet.brotli", )

# full dataset for 33 stocks
df_full = pd.read_parquet("/content/stocks_df_combined_2024_05_07.parquet.brotli", )


In [6]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221142 entries, 0 to 5426
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 288.5+ MB


In [7]:
df_full.keys()

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=202)

In [8]:
# growth indicators (but not future growth)
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

['growth_1d',
 'growth_3d',
 'growth_7d',
 'growth_30d',
 'growth_90d',
 'growth_365d',
 'growth_dax_1d',
 'growth_dax_3d',
 'growth_dax_7d',
 'growth_dax_30d',
 'growth_dax_90d',
 'growth_dax_365d',
 'growth_snp500_1d',
 'growth_snp500_3d',
 'growth_snp500_7d',
 'growth_snp500_30d',
 'growth_snp500_90d',
 'growth_snp500_365d',
 'growth_dji_1d',
 'growth_dji_3d',
 'growth_dji_7d',
 'growth_dji_30d',
 'growth_dji_90d',
 'growth_dji_365d',
 'growth_epi_1d',
 'growth_epi_3d',
 'growth_epi_7d',
 'growth_epi_30d',
 'growth_epi_90d',
 'growth_epi_365d',
 'growth_gold_1d',
 'growth_gold_3d',
 'growth_gold_7d',
 'growth_gold_30d',
 'growth_gold_90d',
 'growth_gold_365d',
 'growth_wti_oil_1d',
 'growth_wti_oil_3d',
 'growth_wti_oil_7d',
 'growth_wti_oil_30d',
 'growth_wti_oil_90d',
 'growth_wti_oil_365d',
 'growth_brent_oil_1d',
 'growth_brent_oil_3d',
 'growth_brent_oil_7d',
 'growth_brent_oil_30d',
 'growth_brent_oil_90d',
 'growth_brent_oil_365d',
 'growth_btc_usd_1d',
 'growth_btc_usd_3d',


In [9]:
# leaving only Volume ==> generate ln(Volume)
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

In [10]:
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

In [11]:
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_PREDICT

['growth_future_5d', 'is_positive_growth_5d_future']

In [12]:
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
TO_DROP

['Year',
 'Date',
 'index_x',
 'index_y',
 'index',
 'Quarter',
 'Adj Close_y',
 'Month',
 'Weekday',
 'Ticker',
 'ticker_type',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close_x',
 'Volume']

In [13]:
# let's define on more custom numerical features
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

In [14]:
# manually defined features
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

In [15]:
# All Supported Ta-lib indicators: https://github.com/TA-Lib/ta-lib-python/blob/master/docs/funcs.md

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

In [16]:
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')

Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [17]:
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']

In [18]:
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

In [19]:
# CHECK: NO OTHER INDICATORS LEFT
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]
OTHER

['growth_future_5d', 'is_positive_growth_5d_future']

In [20]:
df_full.Ticker.nunique()

33

In [21]:
# tickers, min-max date, count of daily observations
df_full.groupby(['Ticker'])['Date'].agg(['min','max','count'])

,min,max,count
Ticker,,,
AAPL,1980-12-12,2024-05-07,10941
ACN,2001-07-19,2024-05-07,5736
AMZN,1997-05-15,2024-05-07,6789
ASML,1995-03-15,2024-05-07,7338
AVGO,2009-08-06,2024-05-07,3713
BHARTIARTL.NS,2002-07-01,2024-05-07,5424
BRK-B,1996-05-09,2024-05-07,7046
CDI.PA,1992-01-27,2024-05-07,8328
GOOG,2004-08-19,2024-05-07,4963


In [22]:
# truncated df_full with 25 years of data (and defined growth variables)
df = df_full[df_full.Date>='2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 203 entries, Open to ln_volume
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 239.7+ MB


In [23]:
# let look at the features count and size:
df[NUMERICAL].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 184 entries, growth_1d to DGS10
dtypes: float64(121), int32(62), int64(1)
memory usage: 214.6 MB


0.2) [Code snippet 1] Generating dummies

In [25]:
# what are the categorical features?
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type']

In [26]:
# dummy variables are not generated from Date and numeric variables
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

In [27]:
# Generate dummy variables (no need for bool, let's have int32 instead)
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')

In [28]:
dummy_variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Data columns (total 55 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   Month_April           182675 non-null  int32
 1   Month_August          182675 non-null  int32
 2   Month_December        182675 non-null  int32
 3   Month_February        182675 non-null  int32
 4   Month_January         182675 non-null  int32
 5   Month_July            182675 non-null  int32
 6   Month_June            182675 non-null  int32
 7   Month_March           182675 non-null  int32
 8   Month_May             182675 non-null  int32
 9   Month_November        182675 non-null  int32
 10  Month_October         182675 non-null  int32
 11  Month_September       182675 non-null  int32
 12  Weekday_0             182675 non-null  int32
 13  Weekday_1             182675 non-null  int32
 14  Weekday_2             182675 non-null  int32
 15  Weekday_3             182675 non-null 

In [29]:
# get dummies names in a list
DUMMIES = dummy_variables.keys().to_list()

In [30]:
# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [31]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 239 entries, growth_1d to ticker_type_US
dtypes: float64(121), int32(117), int64(1)
memory usage: 253.0 MB


0.3) [Code Snippet 2] Correlation analysis

In [32]:
TO_PREDICT

['growth_future_5d', 'is_positive_growth_5d_future']

In [33]:
corr_is_positive_growth_5d_future = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['is_positive_growth_5d_future']

In [34]:
# create a dataframe for an easy way to sort
corr_is_positive_growth_5d_future_df = pd.DataFrame(corr_is_positive_growth_5d_future)

In [35]:
corr_is_positive_growth_5d_future_df.sort_values(by='is_positive_growth_5d_future').head(5)

,is_positive_growth_5d_future
DGS10,-0.036227
gdppot_us_yoy,-0.034185
gdppot_us_qoq,-0.032138
DGS5,-0.030369
growth_brent_oil_90d,-0.028907


In [36]:
corr_is_positive_growth_5d_future_df.sort_values(by='is_positive_growth_5d_future').tail(8)

,is_positive_growth_5d_future
Month_October,0.020550
growth_btc_usd_7d,0.023084
growth_btc_usd_30d,0.027712
growth_future_5d,0.668535
is_positive_growth_5d_future,1.000000
cdl3starsinsouth,NaN
cdlconcealbabyswall,NaN
cdlmathold,NaN


In [37]:
corr_growth_future_5d = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['growth_future_5d']

In [38]:
corr_growth_future_5d_df = pd.DataFrame(corr_growth_future_5d)

In [39]:
corr_growth_future_5d_df.sort_values(by='growth_future_5d').head(5)

,growth_future_5d
growth_7d,-0.050680
growth_3d,-0.049925
growth_brent_oil_365d,-0.041784
roc,-0.040579
rocr,-0.040579


# Question 1 (1 point): Dummies on Month and Week-of-Month

Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable is_positive_growth_5d_future?

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'. Once you've generated the new set of variables, find the most correlated (in absolute value) one with is_positive_growth_5d_future and round it to 3 digits after the comma.

Suggested path to a solution:

[Source] Use this formula to get the week of month for the datetime variable d: (d.day-1)//7+1
Define a new string variable for all month-week_of_month combinations.

Append it to the CATEGORICAL features set. You should have 5 variables treated as CATEGORICAL now: 'Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom'. In the end, you should get 115 dummy features, including 60 (=12*5) week_month_of_week dummies.

Use pandas.get_dummies() to generate dummies.

Use pandas.DataFrame.corr() function (also used in [Code Snippet 1]) to get correlations with is_positive_growth_5d_future, filter out only variables representing the new dummy set, and sort it by absolute values (you can define a new column "abs_corr" in the dataframe with correlations), and find the highest value (among the new dummies features set).
NOTE: new dummies will be used as features in the next tasks, please leave them in the dataset.

In [49]:
# Use this formula to get the week of month for the datetime variable d: (d.day-1)//7+1 Define a new string variable for all month-week_of_month combinations.
df_with_dummies['month_wom'] = (df_with_dummies.Date.dt.day-1)//7+1
df_with_dummies['month_wom'] = df_with_dummies['month_wom'].astype(str)
df_with_dummies.loc[:,'Month'] = df_with_dummies.Month.astype(str)
df_with_dummies['month_wom'] = df_with_dummies['Month'] + '_w' + df_with_dummies['month_wom']


In [48]:
df_with_dummies[['Date', 'month_wom']]

,Date,month_wom
3490,2000-01-03,January_w1
3491,2000-01-04,January_w1
3492,2000-01-05,January_w1
3493,2000-01-06,January_w1
3494,2000-01-07,January_w1
...,...,...
5422,2024-04-30,April_w5
5423,2024-05-02,May_w1
5424,2024-05-03,May_w1
5425,2024-05-06,May_w1


In [50]:
# Append it to the CATEGORICAL features set. You should have 5 variables treated as CATEGORICAL now: 'Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom'. In the end, you should get 115 dummy features, including 60 (=12*5) week_month_of_week dummies.
CATEGORICAL += ['month_wom']

In [51]:
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom']

In [52]:
# Use pandas.get_dummies() to generate dummies.
dummy_variables = pd.get_dummies(df_with_dummies[CATEGORICAL], dtype='int32')

In [54]:
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [63]:
month_wom_cols = [col for col in df_with_dummies.columns if 'month_wom' in col]
month_wom_cols

['month_wom_April_w1',
 'month_wom_April_w2',
 'month_wom_April_w3',
 'month_wom_April_w4',
 'month_wom_April_w5',
 'month_wom_August_w1',
 'month_wom_August_w2',
 'month_wom_August_w3',
 'month_wom_August_w4',
 'month_wom_August_w5',
 'month_wom_December_w1',
 'month_wom_December_w2',
 'month_wom_December_w3',
 'month_wom_December_w4',
 'month_wom_December_w5',
 'month_wom_February_w1',
 'month_wom_February_w2',
 'month_wom_February_w3',
 'month_wom_February_w4',
 'month_wom_February_w5',
 'month_wom_January_w1',
 'month_wom_January_w2',
 'month_wom_January_w3',
 'month_wom_January_w4',
 'month_wom_January_w5',
 'month_wom_July_w1',
 'month_wom_July_w2',
 'month_wom_July_w3',
 'month_wom_July_w4',
 'month_wom_July_w5',
 'month_wom_June_w1',
 'month_wom_June_w2',
 'month_wom_June_w3',
 'month_wom_June_w4',
 'month_wom_June_w5',
 'month_wom_March_w1',
 'month_wom_March_w2',
 'month_wom_March_w3',
 'month_wom_March_w4',
 'month_wom_March_w5',
 'month_wom_May_w1',
 'month_wom_May_w2',
 'm

In [65]:
corr_is_positive_growth_5d_future = df_with_dummies[month_wom_cols+TO_PREDICT].corr()['is_positive_growth_5d_future']

In [68]:
# prompt: create the abs value
abs_corr_is_positive_growth_5d_future = abs(corr_is_positive_growth_5d_future)

In [70]:
# create a dataframe for an easy way to sort
corr_is_positive_growth_5d_future_df = pd.DataFrame(abs_corr_is_positive_growth_5d_future)
corr_is_positive_growth_5d_future_df.sort_values(by='is_positive_growth_5d_future').tail(5)

,is_positive_growth_5d_future
month_wom_October_w5,0.026023
month_wom_March_w4,0.026058
month_wom_September_w3,0.034537
growth_future_5d,0.668535
is_positive_growth_5d_future,1.000000
